In [10]:
import psycopg2
import json
from datetime import datetime as dt
import pandas as pd
import re 
from textblob import TextBlob 

In [11]:
p="dataisgreat"

In [12]:
conn = psycopg2.connect(user = "postgres",
                                  password = f"{p}",
                                  host = "127.0.0.1",
                                  port = "3306",
                                  database = "postgres")

In [13]:
cursor = conn.cursor()

In [14]:
tweet_list = []
followers = []
cursor.execute("select * from tweets")
tweets = cursor.fetchall()
if not tweets:
    print("empty")
for row in tweets:
    for col in row:
        if type(col) is dict:
            tweet_list.append(col['text'])
            followers.append(col['user']['followers_count'])

In [15]:
remove_list = [r'@[A-Za-z0-9]+','https://t.co[A-Za-z0-9./t.co]+','\n','RT :',r'_[A-Za-z0-9]+','  ']

In [16]:
def clean_tweets(remove_list,tweet_list):
    for item in remove_list:
        tweet_list = [re.sub(item,'',tweet) for tweet in tweet_list]
    return tweet_list

In [17]:
cleaned = clean_tweets(remove_list,tweet_list)
cleaned[:5]

['There are so many hilarious comments in You Tubechat on @ show tonight.',
 ' Democrat frontrunner Michael Bloomberg says he wants to make the whole country "behave" while dressed as Mary Poppins.Wel…',
 'The latest COACH!#election2020 #misleadingtweets',
 'THERE IS NO ONE TO VOTE FOR! Centrists demand Valium to get them through #Election2020',
 'RT : Calling all voters!! Are you ready for November?👉🏿RT to spread the word. 🗳️ #Election2020 http…']

In [146]:
# tweet_text = pd.DataFrame(tweet)
# tweet_text.rename(columns = {0: "tweets"}, inplace = True)
# tweet_text.head()

In [18]:
sentiment_objects = []
sentiments = []
for each in cleaned:
    sentiments.append(TextBlob(each).polarity)

    
# sentiment_objects.polarity, sentiment_objects
# print(sentiment_objects[0])
    
print(sentiments[:10])

[0.5, 0.2, 0.5, 0.0, 0.2, 0.0, 0.0, 0.4, 0.5, 0.0]


In [20]:
bernie = []
buttigieg = []
biden = []
warren = []
gender = []
age = []
democrat = []
socialist = []
trump = []
delegates = []
extended = []
length = []
for tweet in cleaned:
    tweet_length = len(tweet)
    length.append(tweet_length)
    if tweet_length > 140:
        extended.append(1)
    else:
        extended.append(0)
    if "delegate" in tweet:
        delegates.append(1)
    else:
        delegates.append(0)
    if "trump" in tweet:
        trump.append(1)
    else:
        trump.append(0)
    if "socialist" in tweet:
        socialist.append(1)
    else:
        socialist.append(0)
    democrat.append(result(tweet.lower(),"democrat"))
    age.append(result(tweet.lower(),"old"))
    gender.append(result(tweet.lower(),"female"))
    warren.append(result(tweet.lower(),"warren"))
    biden.append(result(tweet.lower(),"biden"))
    buttigieg.append(result(tweet.lower(),"buttigieg"))
    bernie.append(result(tweet.lower(),"bernie"))
    
    

In [8]:
def result(tweet,keyword):
    if keyword in tweet:
        return 1
    else:
        return 0

In [21]:
bernie

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,


In [26]:
data = pd.DataFrame({"bernie":bernie,"buttigieg":buttigieg,"biden":biden,"warren":warren,"gender":gender,
             "age":age,"democrat":democrat,"socialist":socialist,"trump":trump,"delegates":delegates,
             "extended":extended,"length":length})

In [31]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=12)


In [62]:
kmeans.fit(data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=12, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [63]:
kmeans.predict(data)

array([3, 8, 6, ..., 8, 8, 8])

In [30]:
len(data[:1850])

1850

In [48]:
test = data[1850:].reset_index(drop=True)

In [84]:
kmeans.predict(data.iloc[75].values.reshape(1,12))

array([8])

In [79]:
(data.iloc[59].values.reshape(1,12)).shape

(1, 12)

In [83]:
data.iloc[75]

bernie         1
buttigieg      0
biden          0
warren         0
gender         0
age            0
democrat       1
socialist      0
trump          0
delegates      0
extended       0
length       122
Name: 75, dtype: int64

In [64]:
for i,row in data.iterrows():
    if row["bernie"] == 1:
        print(i)

26
59
70
75
76
78
80
81
84
96
103
162
191
195
210
214
219
240
245
254
257
262
265
270
306
342
347
350
407
413
417
433
436
450
461
466
489
493
495
526
528
530
536
537
552
578
579
584
586
588
593
605
608
612
616
621
623
629
639
659
661
676
724
746
752
759
761
775
798
803
805
820
823
829
830
836
841
844
870
876
878
881
890
894
895
926
930
947
957
977
988
1008
1009
1028
1032
1054
1065
1067
1070
1082
1093
1097
1098
1104
1105
1113
1122
1127
1144
1147
1148
1152
1159
1167
1173
1180
1182
1196
1198
1199
1204
1207
1231
1234
1236
1237
1238
1241
1245
1254
1268
1271
1272
1274
1286
1287
1294
1297
1301
1320
1321
1324
1328
1333
1336
1337
1358
1389
1446
1448
1469
1505
1516
1537
1545
1548
1552
1568
1576
1598
1602
1638
1645
1668
1671
1721
1724
1729
1742
1748
1762
1767
1768
1776
1793
1817
1916
1931
1937
1946
1951
2004
2019
2054
2117
2124
2129
2147
2151
2157
2163
2190
2209
2227
2234
2248
2263
2264
2302
2303
2304
2305


In [61]:
data

,bernie,buttigieg,biden,warren,gender,age,democrat,socialist,trump,delegates,extended,length
0,0,0,0,0,0,0,0,0,0,0,0,71
1,0,0,0,0,0,0,1,0,0,0,0,123
2,0,0,0,0,0,0,0,0,0,0,0,48
3,0,0,0,0,0,0,0,0,0,0,0,86
4,0,0,0,0,0,0,0,0,0,0,0,100
...,...,...,...,...,...,...,...,...,...,...,...,...
2311,0,0,0,0,0,0,0,0,0,0,0,115
2312,0,0,0,0,0,0,0,0,0,0,0,125
2313,0,0,0,0,0,0,0,0,0,0,0,125
2314,0,0,0,0,0,0,0,0,0,0,0,122
